In [2]:
import pandas as pd
import numpy as np 
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_absolute_error, accuracy_score, log_loss
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.decomposition import PCA
from sklearn.cluster import FeatureAgglomeration
import category_encoders as ce
from numpy.testing import assert_almost_equal
from functools import reduce
import matplotlib.pyplot as plt

In [4]:
## KAGGLE bioresponse https://www.kaggle.com/c/bioresponse#Evaluation

train_url = 'data/train.csv'
test_url = 'data/test.csv' ## iggnoring-- it doesn't have 'Activity' 

df_ = pd.read_csv(train_url)
#df_test = pd.read_csv(test_url) # doesn't have 'Activity'
assert all([x==0 for x in df_.isna().sum().values])
assert all([pd.api.types.is_numeric_dtype(df_[feat]) for feat in df_.columns])
dependent='Activity'

X_train, X_test, y_train, y_test = train_test_split(df_.drop(dependent, axis=1), 
                                                    df_[dependent], 
                                                    train_size=0.8, test_size=0.2)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# df_.dtypes.value_counts()

# #df_.describe()

# df_.mean().array.mean()

(3000, 1776) (751, 1776) (3000,) (751,)


In [5]:

# pipeline1 = make_pipeline(
#     #ce.OneHotEncoder(use_cat_names=True), # no categoricals. 
#     #StandardScaler(), # means are very much near zero anyway, we don't really need this. 
#     LogisticRegression()
# )


# pipeline1.fit(X_train, y_train)
# y_pred = pipeline1.predict(X_test)
# #accuracy_score(y_test, y_pred)
# log_loss(y_test, y_pred)

# scores = cross_val_score(pipeline1, X_train, y_train, cv=10, scoring='neg_log_loss') 

# scores#.mean() # don't run this a lot because it is EXPENSIVE

# # when test_size=0.15, scores.mean is better than accuracy_score... but when test_size=0.2, scores.mena is WORSE. 

###  The above is the "dumbest possible model" we start with. Now we're going to play more. 

In [7]:
# from https://scikit-learn.org/stable/modules/compose.html#pipeline


# # We create the preprocessing pipelines for both numeric and categorical data.
# numeric_features = ['age', 'fare']
# numeric_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='median')),
#     ('scaler', StandardScaler())])

# categorical_features = ['embarked', 'sex', 'pclass']
# categorical_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
#     ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', numeric_transformer, numeric_features),
#         ('cat', categorical_transformer, categorical_features)])

# # Append classifier to preprocessing pipeline.
# # Now we have a full prediction pipeline.
# clf = Pipeline(steps=[('preprocessor', preprocessor),
#                       ('classifier', LogisticRegression(solver='lbfgs'))])



lets dimension-reduction

we'll split the data into two different things, ints and floats. 

a commenter [here](https://stats.stackexchange.com/questions/159705/would-pca-work-for-boolean-binary-data-types) said that a "cosine"ish version of PCA is more appropriate for bool 0,1. 

### also, We're gonna use the heuristic that, for N=#observations and M=#features, N > 5M

In [38]:
ints = df_.drop(dependent, axis=1).select_dtypes(include='int')
floats = df_.drop(dependent, axis=1).select_dtypes(include='float')

print(ints.shape, floats.shape)

B = 5.15

ints_weight = np.divide(ints.shape[1], ints.shape[1] + floats.shape[1] + 1)
floats_weight = np.divide(floats.shape[1], ints.shape[1] + floats.shape[1] + 1)
assert_almost_equal(ints_weight+floats_weight, 1, 3)

n = int(np.divide(df_.shape[0], B))

LogiR = ('logistic_regression', SGDClassifier(loss='log', tol=np.exp(-B/2), max_iter=1234))

estimators_c = [('PCA', PCA(n_components=int(floats_weight * n))), 
                LogiR]

pipe_c = Pipeline(steps=estimators_c)

estimators_d = [('FA', FeatureAgglomeration(n_clusters=int(ints_weight * n), affinity='cosine', linkage='complete')), 
                LogiR]

pipe_d = Pipeline(steps=estimators_d)



(3751, 834) (3751, 942)


In [40]:
ints_train = X_train.select_dtypes(include='int')
ints_test = X_test.select_dtypes(include='int')
floats_train = X_train.select_dtypes(include='float')
floats_test = X_test.select_dtypes(include='float')

print(ints_train.shape, ints_test.shape, floats_train.shape, floats_test.shape)

pipe_c.fit(floats_train, y_train)
y_pred_c = pipe_c.predict_proba(floats_test)

pipe_d.fit(ints_train, y_train)
y_pred_d = pipe_d.predict_proba(ints_test)


# log_loss(y_test, y_pred_c), log_loss(y_test, y_pred_d)

#y_pred_d.shape, y_pred_c.shape, y_test.shape, df_.shape

#y_pred_c, y_pred_d

(3000, 834) (751, 834) (3000, 942) (751, 942)


In [45]:
print(log_loss(y_test, y_pred_c), log_loss(y_test, y_pred_d))

def weighted_pred(p1, y1, p2, y2, epsilon=7): 
    assert_almost_equal(p1+p2, 1, epsilon)
    return p1*y1 + p2*y2

y_pred = weighted_pred(ints_weight, y_pred_d, floats_weight, y_pred_c, epsilon=3)

log_loss(y_test, y_pred) ## an improvement! 

0.7683930364476979 2.2833224568938246


0.7361823955874933

In [51]:
scores_c = cross_val_score(pipe_c, floats_train, y_train, cv=8, scoring='neg_log_loss') 

#scores_d = cross_val_score(pipe_d, ints_train, y_train, cv=5, scoring='neg_log_loss') 

-scores_c.mean()

## ok so when I 
## get rid of the discretes it doesn't get an error. 
# i don't understand SGDClassifier well enough. 
# or feature aggregation. 

# but these are.... not bad??? 

0.6652527725861623

In [52]:
np.arange(b,NCm, a**b), np.logspace(-b**b, b**b, a),

NameError: name 'b' is not defined